## Para este ejercicio uso la API de Spotify y extraigo los new releases de albums en Argentina


In [18]:
!pip install spotipy --upgrade
!pip install -U datasets
!pip install psycopg2-binary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 2.1 MB/s eta 0:00:00a 0:00:01


In [20]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import os
import psycopg2

from pyspark.sql import SparkSession
from pyspark.sql.functions import when, lit, col

In [21]:
env = os.environ

In [22]:
# Configurar las credenciales de cliente de Spotify
client_id = env['SPOTIFY_CLIENT_ID']
client_secret = env['SPOTIFY_CLIENT_SECRET']

# Crear una instancia de SpotifyClientCredentials con las credenciales
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

# Crear una instancia de la clase Spotipy
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [23]:
# Crear lista para almacenar los datos
data = []

# Obtener los nuevos álbumes desde la fecha límite
results = sp.new_releases(country='AR', limit=50)

# Recorrer los nuevos álbumes y obtener los datos requeridos
for album in results['albums']['items']:
        album_fecha = album['release_date']
        artista = album['artists'][0]['name']
        album_nombre = album['name']
        data.append((artista, album_nombre, album_fecha))
        
data

[('Bizarrap', 'Peso Pluma: Bzrp Music Sessions, Vol. 55', '2023-06-01'),
 ('LIT killah', 'Los del Espacio', '2023-06-01'),
 ('Fito Paez', 'EADDA9223', '2023-05-30'),
 ('C.R.O', 'Temor', '2023-06-01'),
 ('Usted Señalemelo', 'Tripolar', '2023-05-29'),
 ('Andrés Calamaro', 'Razzmatazz (En directo Razzmatazz)', '2023-06-02'),
 ('Juan Wauters', 'Wandering Rebel', '2023-06-02'),
 ('Bocho', 'DIAMANTE', '2023-06-01'),
 ('Foo Fighters', 'But Here We Are', '2023-06-02'),
 ('Manuel García', 'Manuel García en Buenos Aires (En Vivo)', '2023-06-02'),
 ('Nicki Nicole', 'ALMA', '2023-05-18'),
 ('Lali', 'LALI', '2023-04-13'),
 ('Cazzu', 'Nena Trampa (Deluxe Edition)', '2023-04-13'),
 ('Lola Indigo', 'EL DRAGÓN', '2023-04-13'),
 ('Connie Isla', 'Isla Mater', '2023-04-13'),
 ('Milo j', '511', '2023-04-13'),
 ('Bhavi', 'ABDUCCION', '2023-04-10'),
 ('Anyi', 'Luz de perla', '2023-04-14'),
 ('Delfina Dib', 'Directo al Cora', '2023-04-14'),
 ('Eladio Carrion', '3MEN2 KBRN', '2023-03-17'),
 ('Manuel Turizo', '

In [24]:
# Postgres and Redshift JDBCs
driver_path = "/home/coder/working_dir/driver_jdbc/postgresql-42.2.27.jre7.jar"

os.environ['PYSPARK_SUBMIT_ARGS'] = f'--driver-class-path {driver_path} --jars {driver_path} pyspark-shell'
os.environ['SPARK_CLASSPATH'] = driver_path

# Create SparkSession 
spark = SparkSession.builder \
        .master("local") \
        .appName("Conexion entre Pyspark y Redshift") \
        .config("spark.jars", driver_path) \
        .config("spark.executor.extraClassPath", driver_path) \
        .getOrCreate()

In [25]:
connection = psycopg2.connect(
    host=env['AWS_REDSHIFT_HOST'],
    port=env['AWS_REDSHIFT_PORT'],
    dbname=env['AWS_REDSHIFT_DBNAME'],
    user=env['AWS_REDSHIFT_USER'],
    password=env['AWS_REDSHIFT_PASSWORD'],
)

In [26]:
# Creacion del DataFrame con los datos 
df = spark.createDataFrame(data, ['Artista', 'Album', 'Fecha_Release'])
df.printSchema()
df.show()

root
 |-- Artista: string (nullable = true)
 |-- Album: string (nullable = true)
 |-- Fecha_Release: string (nullable = true)

+----------------+--------------------+-------------+
|         Artista|               Album|Fecha_Release|
+----------------+--------------------+-------------+
|        Bizarrap|Peso Pluma: Bzrp ...|   2023-06-01|
|      LIT killah|     Los del Espacio|   2023-06-01|
|       Fito Paez|           EADDA9223|   2023-05-30|
|           C.R.O|               Temor|   2023-06-01|
|Usted Señalemelo|            Tripolar|   2023-05-29|
| Andrés Calamaro|Razzmatazz (En di...|   2023-06-02|
|    Juan Wauters|     Wandering Rebel|   2023-06-02|
|           Bocho|            DIAMANTE|   2023-06-01|
|    Foo Fighters|     But Here We Are|   2023-06-02|
|   Manuel García|Manuel García en ...|   2023-06-02|
|    Nicki Nicole|                ALMA|   2023-05-18|
|            Lali|                LALI|   2023-04-13|
|           Cazzu|Nena Trampa (Delu...|   2023-04-13|
|     Lol

In [28]:
# creacion de tabla en redshift           
cursor = connection.cursor()
cursor.execute(f"""
create table if not exists {env['AWS_REDSHIFT_SCHEMA']}.new_releases_spotify (
    artista VARCHAR(30) distkey,
    album VARCHAR(30),
    fecha_release VARCHAR(10)
) sortkey(fecha_release);
""")
connection.commit()
cursor.close()
print("Table created!")

Table created!


In [29]:
#carga el dataframe en la tabla de redshift
df.write \
    .format("jdbc") \
    .option("url", f"jdbc:postgresql://{env['AWS_REDSHIFT_HOST']}:{env['AWS_REDSHIFT_PORT']}/{env['AWS_REDSHIFT_DBNAME']}") \
    .option("dbtable", f"{env['AWS_REDSHIFT_SCHEMA']}.new_releases_spotify") \
    .option("user", env['AWS_REDSHIFT_USER']) \
    .option("password", env['AWS_REDSHIFT_PASSWORD']) \
    .option("driver", "org.postgresql.Driver") \
    .mode("overwrite") \
    .save()

In [30]:
# chequeo con una consulta si los datos estan en la tabla creada en redshift
query = f"select * from {env['AWS_REDSHIFT_SCHEMA']}.new_releases_spotify"
data = spark.read \
    .format("jdbc") \
    .option("url", f"jdbc:postgresql://{env['AWS_REDSHIFT_HOST']}:{env['AWS_REDSHIFT_PORT']}/{env['AWS_REDSHIFT_DBNAME']}") \
    .option("dbtable", f"({query}) as tmp_table") \
    .option("user", env['AWS_REDSHIFT_USER']) \
    .option("password", env['AWS_REDSHIFT_PASSWORD']) \
    .option("driver", "org.postgresql.Driver") \
    .load()

In [31]:
data.printSchema()
data.show()

root
 |-- artista: string (nullable = true)
 |-- album: string (nullable = true)
 |-- fecha_release: string (nullable = true)

+----------------+--------------------+-------------+
|         artista|               album|fecha_release|
+----------------+--------------------+-------------+
|        Bizarrap|Peso Pluma: Bzrp ...|   2023-06-01|
|      LIT killah|     Los del Espacio|   2023-06-01|
|       Fito Paez|           EADDA9223|   2023-05-30|
|           C.R.O|               Temor|   2023-06-01|
|Usted Señalemelo|            Tripolar|   2023-05-29|
| Andrés Calamaro|Razzmatazz (En di...|   2023-06-02|
|    Juan Wauters|     Wandering Rebel|   2023-06-02|
|           Bocho|            DIAMANTE|   2023-06-01|
|    Foo Fighters|     But Here We Are|   2023-06-02|
|   Manuel García|Manuel García en ...|   2023-06-02|
|    Nicki Nicole|                ALMA|   2023-05-18|
|            Lali|                LALI|   2023-04-13|
|           Cazzu|Nena Trampa (Delu...|   2023-04-13|
|     Lol

In [32]:
connection.close()